## Amazon SageMaker HyperParameterTuning 

In this notebook we will demonstrate the following the automated hyperparameter tuning capability with Amazon SageMaker.

### Overview

1. Set up
2. Automatic model tuning (HPT) of XGBoost algorithm for weather prediction
3. Compare Bayesian (default) and Random tuning strtegies

### Step 1 - Set up

##### 1.1 Import libraries

In [1]:
import boto3
import pandas as pd

import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

##### 1.2 Define variables

In [2]:
# set an output path where the trained model will be saved
s3_bucket = 'datascience-environment-notebookinstance--06dc7a0224df'
s3_prefix = 'prepared'
m_prefix = 'hpo'
output_path = 's3://{}/{}/{}/output'.format(s3_bucket, m_prefix, 'xgboost')
content_type = 'text/csv'

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

### Step 2 -  Automatic model tuning of the XGBoost algorithm

#### 2.1 Define parameters necessary for the HPT job

In [3]:
#Objective metric to optimize
objective_metric_name = "validation:rmse"
#Objective type.  Here we want to minimize the objective metric
objective_type = 'Minimize'
#Define the hyperparameter ranges to explore
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
#Total training jobs to be run as part of HPT
max_jobs=10
#Maximum number of training jobs to run in parallel
max_parallel_jobs=2

#### 2.2 Define the estimator and HPT job

In [4]:
# This line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")

In [5]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"5"}

In [6]:
# define the data type and paths to the training and validation datasets
train_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'train'), content_type=content_type, distribution='ShardedByS3Key')
validation_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'validation'), content_type=content_type, distribution='ShardedByS3Key')

In [7]:
#Define the estimator
estimator_hpo = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters, ##Is this needed?
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.12xlarge', 
                                          volume_size=200, # 5 GB 
                                          output_path=output_path)

In [8]:
#Define the Tuner object
tuner = HyperparameterTuner(
    estimator_hpo, 
    objective_metric_name, 
    hyperparameter_ranges, 
    max_jobs=10,  #Total training jobs to be run as part of HPT
    max_parallel_jobs=2, #Maximum number of training jobs to run in parallel
    objective_type = objective_type
)

#### 2.3 Run the tuner job

In [9]:
#Fit method
tuner.fit({'train': train_input, 'validation': validation_input})

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [10]:
tuning_job_name = tuner.latest_tuning_job.name
tuning_job_name

'sagemaker-xgboost-210807-2351'

#### 2.4 Examine the HPT results

In [11]:
##Get results from HPT
tuner_results = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
##Get the results as a dataframe
full_df = tuner_results.dataframe()
##Display results sorted by objective metric values
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 10
{'lowest': 0.478520005941391, 'highest': 0.6521400213241577}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.791758,1.000000,5.0,9.303584,sagemaker-xgboost-210807-2351-010-225ea843,Completed,0.47852,2021-08-08 01:31:16+00:00,2021-08-08 01:50:58+00:00,1182.0
3,0.807994,0.859559,4.0,8.958240,sagemaker-xgboost-210807-2351-007-bfbae99c,Completed,0.50249,2021-08-08 01:10:16+00:00,2021-08-08 01:28:52+00:00,1116.0
8,0.055109,0.165204,8.0,8.342793,sagemaker-xgboost-210807-2351-002-c0683818,Completed,0.50458,2021-08-07 23:54:03+00:00,2021-08-08 00:15:43+00:00,1300.0
7,0.037281,0.753968,8.0,3.775813,sagemaker-xgboost-210807-2351-003-acd79818,Completed,0.51216,2021-08-08 00:15:14+00:00,2021-08-08 00:38:23+00:00,1389.0
9,1.586089,0.602610,3.0,1.451194,sagemaker-xgboost-210807-2351-001-5def3b51,Completed,0.53063,2021-08-07 23:53:45+00:00,2021-08-08 00:11:59+00:00,1094.0
2,0.673939,0.177297,4.0,2.935260,sagemaker-xgboost-210807-2351-008-3ffac53d,Completed,0.55432,2021-08-08 01:10:13+00:00,2021-08-08 01:28:23+00:00,1090.0
6,0.019703,0.104085,1.0,8.741634,sagemaker-xgboost-210807-2351-004-fca96b08,Completed,0.60112,2021-08-08 00:18:34+00:00,2021-08-08 00:34:09+00:00,935.0
1,0.270679,0.871643,8.0,5.661067,sagemaker-xgboost-210807-2351-009-47078b98,Completed,0.64024,2021-08-08 01:31:08+00:00,2021-08-08 01:53:14+00:00,1326.0
5,1.707530,0.676392,10.0,1.465015,sagemaker-xgboost-210807-2351-005-13210913,Completed,0.64992,2021-08-08 00:39:40+00:00,2021-08-08 01:03:27+00:00,1427.0
4,1.484935,0.044220,10.0,9.393024,sagemaker-xgboost-210807-2351-006-9b77dd2f,Completed,0.65214,2021-08-08 00:40:54+00:00,2021-08-08 01:04:42+00:00,1428.0


The tuning job took roughly 2 hours. Best validation rmse observed is 0.42 with max_depth = 6, alpha = 0.82, eta = 0.46, min_child_weight = 3.41.  

What are the best values you observed?

### Step 3 Compare Bayesian (default) and Random tuning strtegies

Now lets try HPO with random strategy.  In the default tuning method above, uses Bayesian Optimization which is a sequential algorithm that learns from past trainings as the tuning job progresses. This highly limits the level of parallelism.

However, with random search you have an option to run all jobs parallel. But a disadvantage is that the random search typically requires running considerably more training jobs to reach a comparable model quality.

Please take into consideration your AWS account limit for the number of instances you can launch in parallel.

In [12]:
#Define the Tuner object
random_tuner = HyperparameterTuner(
    estimator_hpo, 
    objective_metric_name, 
    hyperparameter_ranges, 
    max_jobs=20,  #Total training jobs to be run as part of HPT
    max_parallel_jobs=10, #Maximum number of training jobs to run in parallel
    strategy="Random",
    objective_type = objective_type
)

In [13]:
#Fit method
random_tuner.fit({'train': train_input, 'validation': validation_input})

..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [14]:
tuning_job_name = random_tuner.latest_tuning_job.name
tuning_job_name

'sagemaker-xgboost-210808-0153'

In [15]:
##Get results from HPT
tuner_results = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
##Get the results as a dataframe
full_df = tuner_results.dataframe()
##Display results sorted by objective metric values
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 19
{'lowest': 0.4267500042915344, 'highest': 0.5597400069236755}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,1.902829,0.601191,5.0,4.747202,sagemaker-xgboost-210808-0153-020-57439525,Completed,0.42675,2021-08-08 02:22:55+00:00,2021-08-08 02:42:21+00:00,1166.0
17,0.719768,0.761361,6.0,8.864891,sagemaker-xgboost-210808-0153-004-d496afd0,Completed,0.43923,2021-08-08 01:56:16+00:00,2021-08-08 02:17:05+00:00,1249.0
15,0.596016,0.675929,5.0,9.554632,sagemaker-xgboost-210808-0153-006-9ba28712,Completed,0.46115,2021-08-08 01:56:23+00:00,2021-08-08 02:15:36+00:00,1153.0
19,0.744367,0.893158,7.0,4.708553,sagemaker-xgboost-210808-0153-002-d76adc9e,Completed,0.46333,2021-08-08 01:56:39+00:00,2021-08-08 02:18:23+00:00,1304.0
20,1.460550,0.251139,5.0,3.390766,sagemaker-xgboost-210808-0153-001-045796a8,Completed,0.47879,2021-08-08 01:56:16+00:00,2021-08-08 02:15:46+00:00,1170.0
9,0.445725,0.656241,6.0,9.640180,sagemaker-xgboost-210808-0153-012-a34cca96,Completed,0.47978,2021-08-08 02:15:41+00:00,2021-08-08 02:36:08+00:00,1227.0
6,0.218374,0.237637,5.0,6.574494,sagemaker-xgboost-210808-0153-015-e6e9bbc6,Completed,0.47994,2021-08-08 02:17:25+00:00,2021-08-08 02:37:04+00:00,1179.0
0,0.422439,0.180482,7.0,4.885695,sagemaker-xgboost-210808-0153-021-6229bcab,Completed,0.49743,2021-08-08 02:49:15+00:00,2021-08-08 03:10:42+00:00,1287.0
8,0.980630,0.997615,8.0,8.409500,sagemaker-xgboost-210808-0153-013-4de2cdae,Completed,0.49955,2021-08-08 02:17:10+00:00,2021-08-08 02:40:22+00:00,1392.0
14,0.141610,0.747627,3.0,9.640702,sagemaker-xgboost-210808-0153-007-c1a4821d,Completed,0.53065,2021-08-08 01:56:21+00:00,2021-08-08 02:15:07+00:00,1126.0


The tuning job with random strategy took roughly 60 minutes. Best validation rmse observed is 0.42 with max_depth = 7, alpha = 1.13, eta = 0.22, min_child_weight = 1.02.  

Comparing the times for the tuning with bayesian and random strategy, you see that the random strategy finished about  60 minutes early.  We can see a similar objective value (~0.42), but it took 20 differnet training jobs as opposed to just 10 training jobs with the bayesian strategy.